# Projeto de Previsão de tempo de entrega de e-commerce

---

### Dados utilizados do Olist

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_orders_raw = pd.read_csv('./input/olist_orders_dataset.csv', dtype={'order_id': str})
df_order_reviews_raw = pd.read_csv('./input/olist_order_reviews_dataset.csv', dtype={'order_id': str})


In [ ]:
review = df_order_reviews_raw.copy()
review.dropna(subset=['review_comment_message'], inplace=True)
review.drop(columns=['review_id', 'review_creation_date', 'review_answer_timestamp'], inplace=True)
review['review_comment_message'] = review['review_comment_message'].str.lower()
review = pd.merge(review, df_orders_raw[['order_id', 'customer_id', 'order_delivered_customer_date', 'order_estimated_delivery_date']], on='order_id', how='left')
review['delivery_delay'] = (pd.to_datetime(review['order_delivered_customer_date']) - pd.to_datetime(review['order_estimated_delivery_date'])).dt.days
review['delivery_delay'].sort_values(ascending=False)
review_filtered = review[(review['delivery_delay'] < review['delivery_delay'].quantile(0.99)) & (review['delivery_delay'] > review['delivery_delay'].quantile(0.01))]
review_filtered

In [ ]:
review_filtered = review_filtered.copy()
review_filtered['atrasou'] = review_filtered['delivery_delay'] > 0
atrasos = review_filtered['atrasou'].groupby(review_filtered['review_score']).mean().reset_index()
atrasos['atrasou'] = atrasos['atrasou'] * 100
plt.figure(figsize=(10,6))
sns.barplot(data=atrasos, x='review_score', y='atrasou', palette='magma', hue='review_score', legend=False)
plt.xlabel('Nota de Avaliação')
plt.ylabel('Proporção de Entregas Atrasadas (%)')
plt.title('Proporção de Entregas Atrasadas por Nota de Avaliação')
plt.show()

In [ ]:
df_geolocation_raw = pd.read_csv('./input/olist_geolocation_dataset.csv', dtype={'geolocation_zip_code_prefix': str})

sns.scatterplot(
    y=df_geolocation_raw.geolocation_lat,
    x=df_geolocation_raw.geolocation_lng
)

plt.show()
plt.close()

Latitude e Longitude limites para o Brasil para tratamento dos dados de geolocalização
| Limite  | Valor |
|:-------:|:-----:|
| lat_max | 6.0   |
| lat_min | -34.0 |
| lgn_max | -35.0 |
| lgn_min | -74.0 |

In [ ]:
geo = df_geolocation_raw.copy()

'''
Latitude e Longitude limites para o Brasil para tratamento dos dados de geolocalização
| Limite  | Valor |
|:-------:|:-----:|
| lat_max | 6.0   |
| lat_min | -34.0 |
| lgn_max | -35.0 |
| lgn_min | -74.0 |
'''

geo = geo[(geo.geolocation_lat <= 6.0) & (geo.geolocation_lat >= -34.0)]
geo = geo[(geo.geolocation_lng <= -35.0) & (geo.geolocation_lng >= -74.0)]
display(geo.info())

In [ ]:
sns.scatterplot(
    y=geo.geolocation_lat,
    x=geo.geolocation_lng
)

plt.show()
plt.close()

In [ ]:
df_order_items_raw = pd.read_csv('./input/olist_order_items_dataset.csv', dtype={'order_id': str, 'product_id': str})
df_customers_raw = pd.read_csv('./input/olist_customers_dataset.csv', dtype={'customer_id': str, 'customer_zip_code_prefix': str})
df_sellers_raw = pd.read_csv('./input/olist_sellers_dataset.csv', dtype={'seller_id': str, 'seller_zip_code_prefix': str})
df_products_raw = pd.read_csv('./input/olist_products_dataset.csv', dtype={'product_id': str})

df = pd.merge(
    df_order_items_raw,
    df_orders_raw,
    how='left',
    on='order_id'
)
df = pd.merge(
    df,
    df_customers_raw,
    how='left',
    on='customer_id'
)
df = pd.merge(
    df,
    df_sellers_raw,
    how='left',
    on='seller_id'
)
df = pd.merge(
    df,
    df_products_raw,
    how='left',
    on='product_id'
)

df['product_total_volume_cm3'] = df['product_height_cm'] * df['product_length_cm'] * df['product_width_cm']

df.drop(columns=['customer_unique_id',
                 'product_category_name',
                 'product_name_lenght',
                 'product_description_lenght',
                 'product_photos_qty',
                 'customer_id',
                 'order_approved_at',
                 'order_delivered_carrier_date',
                 'customer_city',
                 'customer_state',
                 'seller_city',
                 'seller_state',
                 'product_length_cm',
                 'product_height_cm',
                 'product_width_cm',
                 'product_id',
                 'order_estimated_delivery_date',
                 'seller_id',
                 'shipping_limit_date'], inplace=True)
df.dropna(inplace=True)
df = df[df['order_status'] == 'delivered']


In [ ]:
df['tempo_entrega_dias'] = (pd.to_datetime(df['order_delivered_customer_date']).dt.date - pd.to_datetime(df['order_purchase_timestamp']).dt.date)
df

In [ ]:
df.columns.to_frame()

In [ ]:
df.describe()